# DNS 서비스

- DNS : IP 주소를 Domain 형식(문자열)로 변환해주는 서비스
- Domain Name : 네트워크에 연결된 모든 시스템은 숫자로 구성된 주소를 갖고 있음.
> 도메인 등록 원칙
- `전 세계적으로 겹치지 않게 하나만 부여`된다.
- 영문자, 숫자, 특수기호 - 를 합쳐 총 37개의 극자로 나타낼 수 있다 (영문자는 대문자소문자 구분 x)

### DNS 관련 파일
- 이 파일들을 바꾸거나 nmcli를 통해 바꿀 수 있음 / vim으로 들어가서 직접 수정 or `nmcli con-add`로 수정도 가능

> `/etc/hosts`
- 인터넷 주소와 호스트 이름의 관계를 설정
- 매우 간단하게 설정가능
- 이 파일의 내용은 로컬 시스템에서만 유효
    - 명령문 예제
        - `cat /etc/hosts` : cat으로 hosts 파일 내용 확인
        - `vim /etc/hosts` : vim으로 hosts 들어가서 직접 수정 / ip 주소 이름
        - `ping servera.example.com` : ping 보내면 ip 주소가 아닌 내가 지정한 이름으로 보내도 잘 가는 것 확인 가능

> `/etc/resolv.conf`
- 호스트가 사용할 DNS 서버의 주소를 설정
- 자신의 호스트가 DNS서버라면 자신의 IP 주소를 적어줘야함
    - 명령문 예제
        - `cat /etc/resolv.conf` : cat으로 resolv.conf 파일 내용 확인
        - `nmcli con add con-name static1 type ethernet ifname eth1 ip4 192.168.56.101/24 gw4 192.168.56.1 ip4 8.8.8.8` : 주소 추가



## Domain Name 구조

<img src="https://qph.cf2.quoracdn.net/main-qimg-b2dd10b04c16667759200b58bb2ed680" height=300 width=500>

- 전 세계적으로 고유한 이름이므로 정해진 규칙/체계에 따라야하며, 임의로 변경이나 생성x
- 도메인 이름은 몇 개의 짧은 영문자를 .로 연결한 계층 구조를 가짐
- 역트리 구조라고 하며 트리 구조의 정점을 root라고 함
- 원래 우리가 쓰는 www.naver~ 요기에는 맨뒤 www.naver.com. 처럼 .이 있다.
- 뒤에서부터 읽는다.

> 1단계. 도메인 or 최상위도메인 TLD(Top Level Domain) : 루트 바로 아래 단계인 도메인 
- 'kr' -> 국가를 나타내는 국가코드 도메인 (ccTLD)
- 'com' 같이 등록인의 목적에 따라 사용되는 일반 도메인
    - `.kr` `.us` `.jp` `.ac` `.fr` 등등

> 2단계. 도메인
- 조직의 속성을 구분하는 도메인
    - `.co` `.or` `.go` `.ac` `.mil` 등등

> 3단계 도메인
- 조직이나 서비스의 이름을 나타내는 도메인 이름으로, 사용자가 원하는 문자열을 사용할 수 있음
    - naver google nate kernerl nobreak 등

> subdoamin
- www blog cafe news 등등

### Domain 동작방식
- 도메인은 계층 구조를 반영한 네임서버(DNS서버)에 저장 및 관리
- 각 네임 서버는 도메인 계층의 일부 영역을 담당하고, 그 영역에 속한 도메인을 관리함

> 재귀 쿼리
- 사용자는 자신이 요청한 DNS 서버에만 요청을 전송하고, 요청에 대한 응답을 수신
- 일반적으로 클라이언트 DNS 서버에 요청할 때 발생
- 경우에 따라 DNS가 다른 DNS에 요청하는 경우에도 사용

> 순환 쿼리
- 클라이언트로부터 받은 이름 요청에 대한 답이 없는 경우 -> 다른 DNS 서버로부터 단계적으로 질의하는 과정을 수행
- 뒤에서부터 차근차근 반복해서 물어보는것
    - kr 물어보고 co 물어보고 naver 물어보고 www 순으로 물어보는 방식

> 정방향 조회 -> 도메인 주소로 ip 찾기
- DNS 서버는 호스트의 이름에 대한 요청을 수신한 후 요청한 이름에 대한 IP 주소를 응답
- 기본적으로 DNS 구조는 계층적 구조에서 도메인의 정보를 가지고 있는 DNS 서버를 찾고, 해당 DNS 서버에서 호스트릐 정보를 확인하도록 설계

> 역방향 조회 -> IP 주소로 도메인 찾기
- IP 주소에 대한 요청을 수신한 후 요청한 IP 주소에 대한 호스트 이름을 응답
- IP 주소로 호스트 및 도메인의 이름을 조회하기 때문에 해당 IP 주소에 대한 레코드를 갖고 있는 DNS 서버를 찾는 것이 불가능
    - 이 문제점을 해결하기 위해 특수 도메인이 설계
    - 이 특수 도메인이 in-addr.arpa
    - in-addr.arpa 도메인에서 다시 각 IP 주소의 대역별로 IP 주소를 조회할 수 있는 DNS 서버가 계층구조로 구성되어 있음
- 역방향 조회를 쓰려면 DNS 서버가 서비스하고 있는 도메인의 각 호스트에 대한 역방향 조회 영역이 필요함.
- 역방향 조회 영역에서는 포인트 레코드(PTR)이 필요함

> DNS 캐시 -> 매번 DNS에 물어보면 과부하가 발생할 수 있어서 캐시로 저장해둔다
- DNS 요청은 시스템이 모르는 이름에 대한 IP 주소가 필요할 때 이루어짐
- 최근 조회했던 내용을 다시 조회 -> 이미 IP주소 알고 있으니까 DNS 요청 발생하지 않게 하기 -> 이렇게 시스템이 기억하고 있는 이름과 IP 주소의 기록 = DNS 캐시
- DNS 캐시가 없다면 주소가 필요할 때마다 매번 DNS 요청을 해야한다. -> 비효율적
- DNS 서비스를 제공하는 서버의 입장에서도 같은 내용 계속 반복 수행해서 과부하 발생 가능
- 위와 같은 일이 발생하지 않게 DNS는 레코드의 수명을 설정해서 한 번 조회한 레코드의 수명이 다하기 전까지는 해당 레코드를 계속 사용하게 함
- DNS 서버도 다른 DNS 서버에서 얻어온 정보를 DNS 캐시로 저장한다.

> 네임 서버의 유형
- 도메인에 대한 DNS 네임 처리를 하기 위해서는 반드시 한대의 네임서버가 필요하다
    - 한대로 DNS 서버 운영은 가능 -> 그러나 다운되면 모든 작업이 중단되기 때문에 이를 대체하기 위한 네임서버 필요
- 주 네임 서버 (Primary Name Server) = Master
    - 1차 네임서버라고도 부르며, 이 네임서버는 영역에 대한 모든 데이터를 가지고 있기 때문에 한명에 대한 모든 권한을 갖고 있는 중요한 서버
- 보조 네임서버 (Secondary Name Server) = Slave
    - 2차 네임서버라고도 부르며, 1차 네임서버가 다운되었을 경우 동작
- 캐시서버 (Cache Server)
    - 직업 입력이 불가능하고 주 서버에서 정보를 가져와서 서비스하는 서버
    - DNS 캐시 저장하는 곳?

> DNS 레코드 -> Zone
- DNS 서버는 DNS 요청을 통해 전달된 특정 호스트의 주소를 응답해줌
- 하지만? DNS 서버는 단순히 특정 이름에 대한 응답만이 아니라 영역(Zone) 도메인 전체에 대한 정보를 제공하기도 함
    - 레코드 종류
    
    |레코드|설명|
    |---|---|
    |A|IPv4|
    |AAAA|IPv6|
    |CNAME|다른 DNS레코드의 별칭 레코드|
    |MX|메일 서버에 대한 정보를 저장|
    |NS|현재 도메인의 네임서버를 저장|
    |PTR|A 레코드와 반대로 특정 IP 주소에 해당하는 도메인의 이름 저장|
    |SOA|영역의 권한 시작 정보(Start of Authority) : 영역을 구성하는 가장 기본 레코드 / 모든 영역은 한 개의 SOA 레코드를 가짐|
    |TXT|자유로운 형식의 텍스트 정보|

> DNS 조회 정보
- `host [option] 도메인이름 [server]`
    - option
        - `-v` : 상세 정보 표시
        - `-t` : 조회할 레코드의 타입 지정
        - `-a` : All 모든 타입의 레코드 조회
        - `-l` : List Mode로 결과 출력 / 일반적인 DNS 레코드 조회 x. 영역 전송 형태로 전송받도록 시도함
- `nslookup [option] 도메인이름 [server]` : 대화형으로 실행됨
- `dig [@server] [option] 도메인이름` : host 명령어에서 상세하게 볼 수 있기 때문에 잘 안씀

## DNS 서버 구성

1. 패키지 설치

|Package 이름|설명|
|---------|-----|
|bind-utils|DNS 쿼리를 위한 유틸리티|
|bind-libs|라이브러리|
|bind-chroot|Chroot 런타임 환경|
|bind-devel|헤더파일 및 라이브러리|
|bind|Bind 패키지|
|unbound|Cache 서버 구성|

2. 네트워크 구성 (고정 ip,DNS 서버 등록, 호스트네임 설정)
- `nmcli con add con-name {이름} type {이더넷} ifname {eth1} ip4 {고정으로 할 IP 주소} gw4 {게이트웨이 IP주소} ipv4 {ip 주소}`

3. 서비스 설정 (/etc/named.conf, /var/named/zone file 설정)
    1. `etc/named.conf` : DNS 서버의 환경을 설정하는 파일
        - option : any, none, any
    2. DNS 서비스 및 방화벽 설정
        - 서비스 시작 및 활성화
            - `systemctl start named`
            - `systemctl enable named`
        - 방화벽 설정
            - `firewall-cmd --add-service=dns --permanent`
            - `firewall-cmd --reload`
        - 설정 확인
            - `nslookup google.com 192.168.56.101`
    3. DNS 영역 구성
        - 도메인에 대한 정보나 도메인 내의 호스트에 대한 정보를 가지고 있는 파일
        - 생성 위치는 /var/named
        - 필수 레코드는 SOA 레코드
        1. DNS 영역 파일 만들기
            - 정방향
                - DNS 영역 파일은 named.empty를 copy해서 생성함
                - `/var/named`위치에서 `cp named.empty 도메인이름.zone`
                - `vim 도메인이름.zone`
                    - 도메인주소 이메일주소 변경
                    - 현재 도메인의 네임서버 정보
                    - 현재 도메인의 기본주소
                    - 현재 도메인 내의 호스트에 대한 레코드 지정
                    - 일반적으로 A 레코드가 호스트의 이름과 IP 주소를 지정
                    - ftp의 별칭을 www로 지정
            - 역방향 정방향과 비슷하지만 각 IP 주소에 대한 PTR 레코드를 설정한다.
        2. 영역 파일의 파일 권한 및 소유권 설정
            - `chmod 660 {정방향파일(또는 역방향)}.zone`
            - `chown :named {정방향파일 또는 역방향}.zone`
    4. 이후 서비스 재시작 및 활성화 해주기
        - `systemctl restart named.service`
    5. `/etc/named.conf 영역 정보 설정`
        - DNS 서버의 연결정보와 함께 BIND를 통해 서비스할 영역에 대한 설정을 저장
        - vi 로 `/etc/named.conf` 편집
            - zone 추가
            - 클라이언트에선 slave 추가